# MTCNN

## Description
Stuff

In [1]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import mtcnn
from os import listdir
from os.path import isfile, join
import time

Using TensorFlow backend.


In [2]:
model = mtcnn.MTCNN()

In [3]:
# draw an image with detected objects
def draw_image_with_boxes(filename, result_list):
    # load the image
    data = pyplot.imread(filename)
    # plot the image
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        # draw the box
        ax.add_patch(rect)
    # show the plot
    pyplot.show()

In [4]:
def singular_test(model, filename):
    pixels = pyplot.imread(filename)
    faces = model.detect_faces(pixels)
    draw_image_with_boxes(filename, faces)

In [8]:
def comprehensive_test(model, filename, results_file):
    list_imgs = [join(filename, f) for f in listdir(filename) if isfile(join(filename, f))]
    results = open(results_file, "w")
    results.close()
    for img in list_imgs:
        pixels = pyplot.imread(img)
        faces = [detected["box"] for detected in model.detect_faces(pixels)]
        with open(results_file, "a") as results:
            results.write(img.split("/")[-1].rstrip() + "\n")
            results.write(str(len(faces)) + '\n')
            for face in faces:
                face = [str(coord) for coord in face]
                results.write(" ".join(face) + "\n")

In [9]:
def compare_results(results_file, expected_file):
    image_faces = {}
    
    with open(expected_file, "r") as reader:
        while True:
            image_dir = reader.readline()
            if not image_dir: 
                break
            image_faces[image_dir] = []
            num_faces = int(reader.readline())
            for i in range(num_faces):
                image_faces[image_dir].append([int(elem) for elem in reader.readline().rstrip().split()])
                
    true_pos = 0
    false_pos = 0
    false_neg = 0
    with open(results_file, "r") as reader:
        while True:
            image_dir = reader.readline()
            
            if not image_dir:
                return true_pos, false_pos, false_neg
            
            expected_images = image_faces[image_dir]
            num_found_faces = int(reader.readline())
            
            for i in range(num_found_faces):
                start_x, start_y, width, height = [int(elem) for elem in reader.readline().split()]
                ## In order to be recognized as a face, it has to be within bounds, therfore
                compatible_index = -1
                
                for i in range(len(expected_images)):
                    expected_face = expected_images[i]
                    if start_x > expected_face[0] - 5 and start_y > expected_face[1] - 5 \
                    and start_x + width < expected_face[2] + 5 and start_y < expected_face[3] + 5:
                        compatible_index = i
                        break
                        
                if compatible_index != -1:
                    true_pos += 1
                    expected_images.pop(compatible_index)
                else:
                    false_pos += 1

            false_neg += len(expected_images)

In [10]:
filename = "./ImageResources/Dataset/WIDER_test/images"
start = time.time()
comprehensive_test(model, filename, "mtcnn_results.txt")
end = time.time()
print("It took " + str(end - start) + " to finish results")

It took 330.1624495983124 to finish results


In [11]:
answers = "./ImageResources/Dataset/WIDER_test/test.txt"
tp, fp, fn = compare_results("mtcnn_results.txt", answers)
print("Correctly found", tp)
print("Incorrectly found", fp)
print("Could not find", fn)

Correctly found 7069
Incorrectly found 1128
Could not find 12936


In [9]:
def comprehensive_simple_test(model, filename):
    list_imgs = [join(filename, f) for f in listdir(filename) if isfile(join(filename, f))]
    counter=0
    for img in list_imgs:
        singular_test(model, img)
        counter+=1
        if counter >100: break

In [12]:
filename = "./ImageResources/Dataset/WIDER_test/cropped_images"
start = time.time()
comprehensive_test(model, filename, "mtcnn_binary_results.txt")
end = time.time()
print("It took " + str(end - start) + " to finish results")

It took 334.6818594932556 to finish results


In [13]:
def compare_bin_results(img_dir, results):
    import cv2
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    with open(results, "r") as reader:
        while True:
            image = reader.readline().rstrip()
            if not image:
                return true_pos, true_neg, false_pos, false_neg
            num_found_faces = int(reader.readline())
            if num_found_faces == 0:
                if image[0:3] == "Pos": 
                    false_neg += 1
                else:
                    true_neg += 1
            for i in range(num_found_faces):
                start_x, start_y, width, height = [int(elem) for elem in reader.readline().split()]
                if image[0:3] == "Pos": #It's a positive image, we check if width and height are closer to the image
                    true_pos += 1
                    #real_height, real_width, _ = cv2.imread(img_dir + image).shape
                    #if abs(real_width - width) < 0.25*real_width and abs(real_height - height) < 0.25*real_height:
                    #    true_pos += 1
                    #else:
                    #    false_pos += 1
                else:
                    false_pos += 1

In [14]:
filename = "./ImageResources/Dataset/WIDER_test/cropped_images"
tp, tn, fp, fn = compare_bin_results(filename+"/", "mtcnn_binary_results.txt")
print("Correctly found", tp)
print("Corerctly could not find", tn)
print("Incorrectly found", fp)
print("Could not find", fn)

Correctly found 6874
Corerctly could not find 9964
Incorrectly found 39
Could not find 3128
